In [ ]:
%reload_ext autoreload
%autoreload 2

from src.file_utils import download_file, decompress_gz,make_dir
import glob
from warcio.archiveiterator import ArchiveIterator
from bs4 import BeautifulSoup
from tqdm import tqdm
from src.parse_warc import extract_japanese_from_warc

import json
import gzip

In [ ]:
import functools

from ja_sentence_segmenter.common.pipeline import make_pipeline
from ja_sentence_segmenter.concatenate.simple_concatenator import concatenate_matching
from ja_sentence_segmenter.normalize.neologd_normalizer import normalize
from ja_sentence_segmenter.split.simple_splitter import split_newline, split_punctuation

split_punc2 = functools.partial(split_punctuation, punctuations=r"。!?")
concat_tail_te = functools.partial(concatenate_matching, former_matching_rule=r"^(?P<result>.+)(て)$", remove_former_matched=False)
segmenter = make_pipeline(normalize, split_newline, concat_tail_te, split_punc2)

text1 = """
私は「あなたの思いに答えられない。他を当たってほしい。」と言われました！呆然として
その場にたたずむしかありませんでしたそれでも私は信じたい！
"""
text1='本題 トップページ こんにちは｡メールでもOKです。'

print(list(segmenter(text1)))

In [ ]:
json_path_list=glob.glob('data/gz/*.gz')
len(json_path_list)

In [ ]:
path="data/jap_dump/crawl-data_CC-MAIN-2023-50_segments_1700679099281.67_warc_CC-MAIN-20231128083443-20231128113443-00000.warc_jap.gz.gz"
# gzipで圧縮されたJSONファイルを読み込み
with gzip.open(path, 'rt', encoding='utf-8') as f:
    data_loaded = json.load(f)


In [ ]:
import kenlm
import sentencepiece
import unicodedata


class PerplexityChecker:
    def __init__(self, arpa_path, sp_path):
        self.ken_model = kenlm.LanguageModel(arpa_path)
        self.sp = sentencepiece.SentencePieceProcessor()
        self.sp.load(sp_path)

    def __call__(self, inp):
        text = unicodedata.normalize('NFD', inp)
        toks = self.sp.encode(text, out_type=str)
        sentence = " ".join(toks)
        ppl = self.ken_model.perplexity(sentence)
        return ppl




In [ ]:
checker=PerplexityChecker("data/lm_sp/ja.arpa.bin",
                          "data/lm_sp/ja.sp.model",
                          )

In [ ]:
record=data_loaded[17]

new_lines=[]
current_line=""
#old_line=""
dup_n_threshold=100
for line in record["text"]:
    line=line[0].strip()
    #if line in new_lines:
    if line in new_lines[-dup_n_threshold:]:
        #print("dup:",line)
        continue
    old_line=line
    print(line)
    joint_text=current_line+line
    sep_text=current_line+"\n"+line

    joint_ppl=checker(joint_text)
    sep_ppl=checker(sep_text)
    
    if joint_ppl<sep_ppl:
        current_line=joint_text
    else:
        new_lines.append(current_line)
        current_line=line


In [ ]:
t1="""入力、または「
GREE"""
t2="""入力、または「GREE"""

checker(t1),checker(t2)

In [ ]:
"お問い合わせ"=="お問い合わせ"